- 将好几个 operations fuse 成一个 operation 进而减轻 memory 存取的 loading
- Operation Fusion 的基本思想是将多个可以连续执行的操作组合成一个复合操作，从而减少数据在显存和计算单元之间的传输次数。

## operation fused 与 kernel

- GPU 内核是运行在 GPU 上的一段程序代码，用于执行特定的计算任务。在深度学习中，常见的 GPU 内核包括矩阵乘法、加法、激活函数等操作。每个内核负责一个特定的操作，并在 GPU 的大量计算单元上并行执行。
- 操作融合是将多个可以连续执行的操作组合成一个复合操作，从而减少中间数据的读写，提高计算效率。操作融合的目的是将多个独立的操作合并成一个内核，从而减少内存访问的次数，提高整体计算性能。

```
# 独立的内核调用
a = x + y  # 内核1
b = a * z  # 内核2
c = torch.relu(b)  # 内核3

# 优化后的内核（操作融合为一个内核）
# 定义操作融合的内核（使用 TorchScript）
@torch.jit.script
def fused_kernel(x, y, z):
    a = x + y
    b = a * z
    c = torch.relu(b)
    return c
```

## 一个示例

- 三步操作
    - A = X + Y
        - 第一步：从显存读取 X 和 Y，计算 A，然后将 A 写回显存。
    - B = A * Z
        - 第二步：从显存读取 A 和 Z，计算 B，然后将 B 写回显存。
    - C = relu(B)
        - 第三步：从显存读取 B，计算 C，然后将 C 写回显存。
- 通过 Operation Fusion，可以将这些操作融合成一个复合操作，如下：
    - 一步：从显存读取 X、Y 和 Z，直接计算 C 而不需要中间存储 A 和 B。

In [1]:
import torch
import time
from tqdm import tqdm

# 初始化数据
x = torch.rand(20000, 20000).cuda()
y = torch.rand(20000, 20000).cuda()
z = torch.rand(20000, 20000).cuda()

In [8]:
# 定义未优化的操作
def unoptimized_operations(x, y, z):
    a = x + y
    b = a * z
    c = torch.relu(b)
    return c

# 进行多次测试以计算平均时间
num_runs = 100
total_time = 0

for _ in tqdm(range(num_runs)):
    start_time = time.time()
    c = unoptimized_operations(x, y, z)
    torch.cuda.synchronize()  # 确保所有 CUDA 操作完成
    total_time += time.time() - start_time

total_time / num_runs

100%|██████████| 100/100 [00:01<00:00, 71.47it/s]


0.013847415447235107

In [7]:
@torch.jit.script
def optimized_operations(x, y, z):
    a = x + y
    b = a * z
    c = torch.relu(b)
    return c

# 进行多次测试以计算平均时间
total_time = 0

for _ in tqdm(range(num_runs)):
    start_time = time.time()
    c = optimized_operations(x, y, z)
    torch.cuda.synchronize()  # 确保所有 CUDA 操作完成
    total_time += time.time() - start_time

total_time / num_runs

100%|██████████| 100/100 [00:00<00:00, 115.69it/s]


0.00855867862701416